<a href="https://colab.research.google.com/github/samiravieira16-ui/projeto-dengue-mortalidade/blob/main/Execu%C3%A7%C3%A3o_dos_projetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Leitura dos dados Convertidos**
## Instalações e autorizações necessarias para o acesso do local de consulta.

In [ ]:
# 0) Montar o Google Drive (autorize quando pedir) essa celula é necessaria somente no Colab.
from google.colab import drive
drive.mount('/content/drive')

# 0) Caminho para a pasta onde estão os .parquet após adicionar atalho à "Meu Drive".
DATA_DIR = "/content/drive/MyDrive/Projeto Integrado (2º Semestre)/Parquet"   # <- altere conforme seu MyDrive

Mounted at /content/drive


# **Cabeçalho do Dataset**
---
Verificando se as colunas são totalmente iguais

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# DATA_DIR = "/data/Parquet"   # <- altere conforme o caminho dos arquivos parquet
DATA_DIR = "/content/drive/MyDrive/Projeto Integrado (2º Semestre)/Parquet"
specific_parquet_file = f"{DATA_DIR}/DENGBR25.parquet"

# Leitura do arquivo .parquet
df_specific = pd.read_parquet(specific_parquet_file)

print(f"Primeiras 2 linhas do arquivo {specific_parquet_file}:")
print(df_specific.head(1))


Primeiras 2 linhas do arquivo /content/drive/MyDrive/Projeto Integrado (2º Semestre)/Parquet/DENGBR25.parquet:
  TP_NOT ID_AGRAVO  DT_NOTIFIC SEM_NOT NU_ANO SG_UF_NOT ID_MUNICIP ID_REGIONA ID_UNIDADE  DT_SIN_PRI SEM_PRI ANO_NASC NU_IDADE_N CS_SEXO CS_GESTANT CS_RACA CS_ESCOL_N SG_UF ID_MN_RESI ID_RG_RESI ID_PAIS DT_INVEST ID_OCUPA_N FEBRE MIALGIA CEFALEIA EXANTEMA VOMITO NAUSEA DOR_COSTAS CONJUNTVIT ARTRITE ARTRALGIA PETEQUIA_N LEUCOPENIA  LACO DOR_RETRO DIABETES HEMATOLOG HEPATOPAT RENAL HIPERTENSA ACIDO_PEPT AUTO_IMUNE DT_CHIK_S1 DT_CHIK_S2 DT_PRNT RES_CHIKS1 RES_CHIKS2 RESUL_PRNT DT_SORO RESUL_SORO DT_NS1 RESUL_NS1 DT_VIRAL RESUL_VI_N DT_PCR RESUL_PCR_ SOROTIPO HISTOPA_N IMUNOH_N HOSPITALIZ DT_INTERNA    UF MUNICIPIO TPAUTOCTO COUFINF COPAISINF COMUNINF CLASSI_FIN CRITERIO DOENCA_TRA CLINC_CHIK EVOLUCAO DT_OBITO DT_ENCERRA ALRM_HIPOT ALRM_PLAQ ALRM_VOM ALRM_SANG ALRM_HEMAT ALRM_ABDOM ALRM_LETAR ALRM_HEPAT ALRM_LIQ DT_ALRM GRAV_PULSO GRAV_CONV GRAV_ENCH GRAV_INSUF GRAV_TAQUI GRAV_EXT

# Analise dos ultimos 5 anos de dados

Os dados apresentam inconsistências entre as colunas, existe nomes diferentes de colunas para um mesmo significado.
Dessa forma, foi feito a paridade entre as colunas e estabelecendo os nomes individualmente das colunas para o seguintes anos e corrigindo:

### DENGBR21 é igual ao DENGBR22
---
### DENGBR23 é igual ao DENGBR24
---
### DENGBR25 é diferente dos demais
---

In [ ]:
import pandas as pd
import numpy as np
import os
import gc # Garbage Collector

# ==============================================================================
# 1. CONFIGURAÇÃO E LISTA MESTRA DE COLUNAS
# ==============================================================================
# DATA_DIR = "/data/Parquet"   # <- altere conforme o caminho dos arquivos parquet
DATA_DIR = "/content/drive/MyDrive/Projeto Integrado (2º Semestre)/Parquet"
OUTPUT_DIR = os.path.join(DATA_DIR, "Dataset_Limpo_Final")

# Cria a pasta de saída se não existir
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Sua lista de colunas OFICIAL
COLUNAS_OFICIAIS = [
    'TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT',
    'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'UF', 'MUNICIPIO', 'ID_MN_RESI',
    'ID_RG_RESI', 'SG_UF', 'ID_PAIS', 'DT_SIN_PRI', 'SEM_PRI', 'ANO_NASC',
    'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'ID_OCUPA_N',
    'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS',
    'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO',
    'DOR_RETRO', 'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
    'ACIDO_PEPT', 'AUTO_IMUNE', 'DT_INVEST', 'DT_CHIK_S1', 'DT_CHIK_S2', 'DT_PRNT',
    'RES_CHIKS1', 'RES_CHIKS2', 'RESUL_PRNT', 'DT_SORO', 'RESUL_SORO', 'DT_NS1',
    'RESUL_NS1', 'DT_VIRAL', 'RESUL_VI_N', 'DT_PCR', 'RESUL_PCR_', 'SOROTIPO',
    'HISTOPA_N', 'IMUNOH_N', 'CLASSI_FIN', 'CRITERIO', 'TPAUTOCTO', 'COUFINF',
    'COPAISINF', 'COMUNINF', 'DOENCA_TRA', 'CLINC_CHIK', 'EVOLUCAO', 'DT_OBITO',
    'DT_ENCERRA', 'HOSPITALIZ', 'DT_INTERNA', 'ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM',
    'ALRM_SANG', 'ALRM_HEMAT', 'ALRM_ABDOM', 'ALRM_LETAR', 'ALRM_HEPAT', 'ALRM_LIQ',
    'DT_ALRM', 'GRAV_PULSO', 'GRAV_CONV', 'GRAV_ENCH', 'GRAV_INSUF', 'GRAV_TAQUI',
    'GRAV_EXTRE', 'GRAV_HIPOT', 'GRAV_HEMAT', 'GRAV_MELEN', 'GRAV_METRO',
    'GRAV_SANG', 'GRAV_AST', 'GRAV_MIOC', 'GRAV_CONSC', 'GRAV_ORGAO', 'DT_GRAV',
    'MANI_HEMOR', 'EPISTAXE', 'GENGIVO', 'METRO', 'PETEQUIAS', 'HEMATURA', 'SANGRAM',
    'LACO_N', 'PLASMATICO', 'EVIDENCIA', 'PLAQ_MENOR', 'CON_FHD', 'COMPLICA',
    'TP_SISTEMA', 'NDUPLIC_N', 'DT_DIGITA', 'CS_FLXRET', 'FLXRECEBI', 'MIGRADO_W'
]

# Colunas que podem ser transformadas em CATEGORIA para economizar RAM
COLS_CATEGORIA = [
    'TP_NOT', 'SG_UF_NOT', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N',
    'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS',
    'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO',
    'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'AUTO_IMUNE',
    'CLASSI_FIN', 'EVOLUCAO', 'CRITERIO', 'HOSPITALIZ'
]

# ==============================================================================
# 2. FUNÇÃO DE OTIMIZAÇÃO
# ==============================================================================
def processar_e_salvar(ano, arquivo_entrada, pasta_saida):
    print(f"--> Processando Ano 20{ano}...", end=" ")

    try:
        # A. Ler arquivo
        df = pd.read_parquet(arquivo_entrada)

        # B. Padronizar Colunas (Reindex)
        # Cria colunas faltantes com NaN e descarta as inúteis
        df = df.reindex(columns=COLUNAS_OFICIAIS)

        # C. Adicionar Ano de Origem
        df['ANO_ORIGEM'] = int("20" + ano)

        # D. Otimização de Memória (Downcasting)
        # 1. Converter Float64 -> Float32
        cols_float = df.select_dtypes(include=['float64']).columns
        df[cols_float] = df[cols_float].astype('float32')

        # 2. Converter Object -> Category
        for col in COLS_CATEGORIA:
            if col in df.columns:
                # Converte apenas se não for tudo nulo
                if df[col].notna().any():
                    df[col] = df[col].astype('category')

        # E. Salvar Imediatamente (Partitioning)
        # Salvamos como uma "fatia" do dataset final
        caminho_saida = os.path.join(pasta_saida, f"part_dengue_20{ano}.parquet")
        df.to_parquet(caminho_saida, index=False)

        print(f"SALVO! (Memória liberada)")

        # F. Limpeza de Memória Explícita
        del df
        gc.collect() # Chama o GC
        return True

    except Exception as e:
        print(f"ERRO: {e}")
        return False

# ==============================================================================
# 3. EXECUÇÃO DO FLUXO
# ==============================================================================
ANOS = ['21', '22', '23', '24', '25']

print(f"Iniciando pipeline de unificação com economia de memória.")
print(f"Os arquivos processados serão salvos em: {OUTPUT_DIR}\n")

for ano in ANOS:
    arquivo = f"{DATA_DIR}/DENGBR{ano}.parquet"
    if os.path.exists(arquivo):
        processar_e_salvar(ano, arquivo, OUTPUT_DIR)
    else:
        print(f"AVISO: Arquivo 20{ano} não encontrado.")

print("-" * 50)
print("PROCESSO CONCLUÍDO!\n")
print("Para ler o dataset completo sem estourar a memória, use:")
print(f"df_completo = pd.read_parquet('{OUTPUT_DIR}')")

Iniciando pipeline de unificação com economia de memória.
Os arquivos processados serão salvos em: /content/drive/MyDrive/Projeto Integrado (2º Semestre)/Parquet/Dataset_Unificado_Otimizado

--> Processando Ano 2021... SALVO! (Memória liberada)
--> Processando Ano 2022... SALVO! (Memória liberada)
--> Processando Ano 2023... SALVO! (Memória liberada)
--> Processando Ano 2024... 

# Trabalhando com os Dataset
---
## Reestruturando de acordo com as novas regras das colunas.
---

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import gc

# ==============================================================================
# 1. CONFIGURAÇÃO DE CAMINHOS
# ==============================================================================
DATA_DIR = "/content/drive/MyDrive/Projeto Integrado (2º Semestre)/Parquet"
INPUT_DIR = os.path.join(DATA_DIR, "Dataset_Limpo_Final")
OUTPUT_DIR = os.path.join(DATA_DIR, "Dataset_Pronto_Para_Uso") # Pasta nova

# Cria a pasta de saída
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Lendo de: {INPUT_DIR}")
print(f"Escrevendo em: {OUTPUT_DIR}")

# ==============================================================================
# 2. DICIONÁRIO DE RENOMEAÇÃO (Obrigatório)
# ==============================================================================
dict_renomear = {
    # Identificação
    'TP_NOT': 'Tipo_Notificacao', 'ID_AGRAVO': 'Codigo_Doenca', 'DT_NOTIFIC': 'Data_Notificacao',
    'NU_ANO': 'Ano_Notificacao', 'SG_UF_NOT': 'UF_Notificacao', 'UF': 'UF_Residencia',
    'MUNICIPIO': 'Municipio_Residencia', 'ID_MN_RESI': 'Cod_Municipio_Residencia', 'ID_PAIS': 'Pais_Residencia',
    # Paciente
    'DT_SIN_PRI': 'Data_Inicio_Sintomas', 'NU_IDADE_N': 'Idade_Codificada',
    'CS_SEXO': 'Sexo', 'CS_RACA': 'Raca_Cor', 'CS_ESCOL_N': 'Escolaridade',
    'CS_GESTANT': 'Gestante', 'ID_OCUPA_N': 'Ocupacao',
    # Sintomas
    'FEBRE': 'Sintoma_Febre', 'MIALGIA': 'Sintoma_DorCorpo', 'CEFALEIA': 'Sintoma_DorCabeca',
    'EXANTEMA': 'Sintoma_Manchas', 'VOMITO': 'Sintoma_Vomito', 'NAUSEA': 'Sintoma_Nausea',
    'DOR_COSTAS': 'Sintoma_DorCostas', 'CONJUNTVIT': 'Sintoma_Conjuntivite',
    'ARTRITE': 'Sintoma_Artrite', 'ARTRALGIA': 'Sintoma_DorArticulacao',
    'PETEQUIA_N': 'Sintoma_Petequias', 'LEUCOPENIA': 'Sintoma_Leucopenia',
    'LACO': 'Sintoma_ProvaLaco', 'DOR_RETRO': 'Sintoma_DorRetroorbital',
    # Comorbidades
    'DIABETES': 'Comorb_Diabetes', 'HEMATOLOG': 'Comorb_Hematolog', 'HEPATOPAT': 'Comorb_Hepatopat',
    'RENAL': 'Comorb_Renal', 'HIPERTENSA': 'Comorb_Hipertensao',
    'ACIDO_PEPT': 'Comorb_AcidoPeptica', 'AUTO_IMUNE': 'Comorb_AutoImune',
    # Alarme e Gravidade
    'ALRM_HIPOT': 'Alarme_Hipotensao', 'ALRM_PLAQ': 'Alarme_QuedaPlaquetas',
    'ALRM_VOM': 'Alarme_VomitoPersistente', 'ALRM_SANG': 'Alarme_SangramentoMucosa',
    'ALRM_HEMAT': 'Alarme_AumentoHematocrito', 'ALRM_ABDOM': 'Alarme_DorAbdominal',
    'ALRM_LETAR': 'Alarme_Letargia', 'GRAV_PULSO': 'Gravidade_PulsoDebil',
    'GRAV_CONV': 'Gravidade_PA_Convergente', 'GRAV_ENCH': 'Gravidade_EnchimentoLento',
    'GRAV_INSUF': 'Gravidade_InsufRespiratoria', 'GRAV_TAQUI': 'Gravidade_Taquicardia',
    'GRAV_EXTRE': 'Gravidade_ExtremidadesFrias',
    # Desfecho
    'CLASSI_FIN': 'Classificacao_Final', 'CRITERIO': 'Criterio_Confirmacao',
    'EVOLUCAO': 'Desfecho_Caso', 'DT_OBITO': 'Data_Obito',
    'DT_ENCERRA': 'Data_Encerramento', 'HOSPITALIZ': 'Houve_Hospitalizacao',
    'DT_INTERNA': 'Data_Internacao'
}

# ==============================================================================
# 3. FUNÇÃO DE PROCESSAMENTO (1 Arquivo por vez)
# ==============================================================================
def processar_lote(arquivo_entrada, arquivo_saida):
    print(f"--> Processando: {os.path.basename(arquivo_entrada)}...", end=" ")

    try:
        # A. Carrega apenas este arquivo
        df = pd.read_parquet(arquivo_entrada)

        # B. Renomeia IMEDIATAMENTE (Resolve o erro KeyError)
        df = df.rename(columns=dict_renomear)

        # C. Dados regras do PDF (Cálculos)
        # Datas
        cols_data = ['Data_Notificacao', 'Data_Inicio_Sintomas', 'Data_Obito', 'Data_Encerramento']
        for col in cols_data:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')

        # Idade Real
        if 'Idade_Codificada' in df.columns:
            df['Idade_Codificada'] = pd.to_numeric(df['Idade_Codificada'], errors='coerce')
            condicoes = [df['Idade_Codificada'].isna(), df['Idade_Codificada'] >= 4000]
            escolhas = [np.nan, df['Idade_Codificada'] - 4000]
            df['Idade_Em_Anos'] = np.select(condicoes, escolhas, default=0)

        # Classificação Final
        if 'Classificacao_Final' in df.columns:
            df['Classificacao_Final'] = pd.to_numeric(df['Classificacao_Final'], errors='coerce')
            mapa = {10: 'Dengue', 11: 'Dengue com Sinais de Alarme', 12: 'Dengue Grave', 5: 'Descartado'}
            df['Descricao_Classificacao'] = df['Classificacao_Final'].map(mapa)

        # D. Otimização de Memória
        cols_cat = ['Sexo', 'Raca_Cor', 'UF_Notificacao', 'Descricao_Classificacao', 'Desfecho_Caso']
        for col in cols_cat:
            if col in df.columns:
                df[col] = df[col].astype('category')

        # E. Salva o resultado limpo
        df.to_parquet(arquivo_saida, index=False)
        print("OK (Salvo).")

        # F. Limpeza de RAM
        del df
        gc.collect()

    except Exception as e:
        print(f"FALHA: {e}")

# ==============================================================================
# 4. LOOP DE EXECUÇÃO
# ==============================================================================
# Pega os arquivos parciais
arquivos = sorted(glob.glob(os.path.join(INPUT_DIR, "*.parquet")))
# Filtra para não pegar arquivo final antigo, se houver
arquivos = [f for f in arquivos if "df_completo" not in f]

if not arquivos:
    print("ERRO: Nenhum arquivo encontrado para processar.")
else:
    print(f"Iniciando processamento de {len(arquivos)} arquivos.\n")

    for arq in arquivos:
        nome_base = os.path.basename(arq)
        # Cria nome de saída: "dengue_limpo_part_2021.parquet"
        arq_saida = os.path.join(OUTPUT_DIR, f"dengue_limpo_{nome_base}")
        processar_lote(arq, arq_saida)

    print("\n" + "="*50)
    print("PROCESSO FINALIZADO!")
    print(f"Todos os arquivos processados estão na pasta:\n{OUTPUT_DIR}")
    print("="*50)
    print("COMO USAR AGORA:")
    print("Não tente ler um arquivo só. Leia a PASTA inteira:")
    print(f"df = pd.read_parquet('{OUTPUT_DIR}')")

# Teste se o resultando atende?!

In [ ]:
import pandas as pd
import os
import glob
import gc

# ==============================================================================
# CONFIGURAÇÃO
# ==============================================================================
DATA_DIR = "/content/drive/MyDrive/Projeto Integrado (2º Semestre)/Parquet"
PASTA_DOS_DADOS = os.path.join(DATA_DIR, "Dataset_Pronto_Para_Uso")

print(f"📂 Lendo dados da pasta: {PASTA_DOS_DADOS}...")

# ==============================================================================
# 1. CARREGAMENTO ROBUSTO (Bypass do erro PyArrow)
# ==============================================================================
try:
    # Passo 1: Listar os arquivos na pasta manualmente
    arquivos_parquet = sorted(glob.glob(os.path.join(PASTA_DOS_DADOS, "*.parquet")))

    if not arquivos_parquet:
        raise FileNotFoundError("Nenhum arquivo encontrado na pasta!")

    print(f"   Encontrados {len(arquivos_parquet)} arquivos. Lendo um a um...")

    lista_dfs = []

    # Passo 2: Ler individualmente (Isso evita o erro de Schema Mismatch)
    for arquivo in arquivos_parquet:
        nome = os.path.basename(arquivo)
        print(f"   -> Carregando: {nome}...", end=" ")

        # Lê o arquivo isolado
        df_temp = pd.read_parquet(arquivo)
        lista_dfs.append(df_temp)
        print("OK.")

    # Passo 3: O Pandas junta e resolve os conflitos de tipo (Null vs String)
    print("   -> Consolidando tabelas (Pandas Concat)...")
    df_final = pd.concat(lista_dfs, ignore_index=True)

    # Limpa memória auxiliar
    del lista_dfs
    gc.collect()

    print(f"✅ SUCESSO! Dataset carregado na memória.")
    print(f"📊 Dimensões Finais: {df_final.shape[0]:,} linhas x {df_final.shape[1]} colunas")

except Exception as e:
    print(f"\n❌ ERRO CRÍTICO: {e}")
    raise

# ==============================================================================
# 2. VERIFICAÇÃO RÁPIDA (Só roda se o carregamento funcionar)
# ==============================================================================
if 'df_final' in locals():
    print("\n" + "="*50)
    print("🔍 CHECKLIST DE INTEGRIDADE")
    print("="*50)

    # A. Anos presentes
    if 'Ano_Notificacao' in df_final.columns:
        print("\n1. CASOS POR ANO:")
        print(df_final['Ano_Notificacao'].value_counts().sort_index())

    # B. Teste de Idade
    if 'Idade_Em_Anos' in df_final.columns:
        print("\n2. AMOSTRA DE IDADE REAL:")
        print(df_final['Idade_Em_Anos'].dropna().sample(5))

    # C. Teste de Gravidade
    if 'Descricao_Classificacao' in df_final.columns:
        print("\n3. GRAVIDADE (Amostra):")
        print(df_final['Descricao_Classificacao'].value_counts().head())

📂 Lendo dados da pasta: /content/drive/MyDrive/Projeto Integrado (2º Semestre)/Parquet/Dataset_Pronto_Para_Uso...
   Encontrados 5 arquivos. Lendo um a um...
   -> Carregando: dengue_limpo_part_dengue_2021.parquet... OK.
   -> Carregando: dengue_limpo_part_dengue_2022.parquet... OK.
   -> Carregando: dengue_limpo_part_dengue_2023.parquet... OK.
   -> Carregando: dengue_limpo_part_dengue_2024.parquet... 